In [111]:
import os;
import json;
import builtins
import pandas as pd;

dir = '/home/chesetti/Repos/learned_merge_cleanup'
testcase = 'pgm_vs_flat/join_fb_1_1'
test_dir = os.path.join(dir, testcase)
results_dir = os.path.join(test_dir, 'outputs', 'results')
csv_dir = os.path.join(test_dir, 'csv')
os.makedirs(csv_dir, exist_ok=True)

runs = [os.path.join(results_dir, run) for run in os.listdir(results_dir)]
print(runs)

FileNotFoundError: [Errno 2] No such file or directory: '/home/chesetti/Repos/learned_merge_cleanup/pgm_vs_flat/join_fb_1_1/outputs/results'

In [ ]:
# Walk all the json files and put them in a dataframe

test_results = []
for run in runs:
    for test_result_file in os.listdir(run):
        json_file = builtins.open(os.path.join(run, test_result_file))
        test_result = json.load(json_file)
        test_result['run'] = run
        test_results.append(test_result)
        json_file.close()
test_dataframe = pd.json_normalize(test_results)
print(test_dataframe.columns)

        

Index(['command', 'run', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.outer_disk_fetch', 'result.outer_disk_fetch_size',
       'result.outer_total_bytes_fetched', 'spec.algo', 'spec.algo_name',
       'spec.check_checksum', 'spec.common_key', 'spec.index.search',
       'spec.index.type', 'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk'],
      dtype='object')


# Reports

### Overall Duration

In [ ]:
overall_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.duration_ns', aggfunc='median')
overall_duration.to_csv(os.path.join(csv_dir, 'duration_sec.csv'))
print(overall_duration.columns)
print(overall_duration.index)
print(overall_duration)
throughput = overall_duration
print(throughput[['flatpgm256', 'pgm256', 'flatpgm1024', 'pgm1024', 'flatpgm2048', 'pgm2048']].to_markdown())
'''
for column in throughput.columns:
    throughput[column] = (200_000_000/throughput.index) * (1000_000_000.0 / throughput[column])
    throughput[column +"_sj-rel"] = (throughput[column] - throughput["sj"] / throughput["sj"]) * 100.0
print(throughput)
overall_duration[["btree2048", "btree1024", "btree256", "sj"]].plot() 
overall_duration[["pgm128", "pgm512", "pgm1024", "sj"]].plot() 
'''

Index(['flatpgm1024', 'flatpgm2048', 'flatpgm256', 'pgm1024', 'pgm2048',
       'pgm256', 'sj'],
      dtype='object', name='spec.algo_name')
Index([1, 5, 10, 50, 100, 500, 1000], dtype='int64', name='spec.common_key')
spec.algo_name   flatpgm1024  flatpgm2048   flatpgm256      pgm1024  \
spec.common_key                                                       
1                916012932.0  985156154.0  837804812.5  979792843.0   
5                226729518.0  232052062.5  213930525.0  239461365.0   
10               131759663.5  131993212.0  128813413.0  139699101.5   
50                53104401.0   52924093.0   50480582.5   52725617.0   
100               37887636.0   41648986.5   42227045.5   35764968.5   
500               26888288.0   24998891.0   20598737.0   23332842.0   
1000              22599161.0   26367658.5   11140243.5   19437566.0   

spec.algo_name        pgm2048       pgm256           sj  
spec.common_key                                          
1                1.014986

'\nfor column in throughput.columns:\n    throughput[column] = (200_000_000/throughput.index) * (1000_000_000.0 / throughput[column])\n    throughput[column +"_sj-rel"] = (throughput[column] - throughput["sj"] / throughput["sj"]) * 100.0\nprint(throughput)\noverall_duration[["btree2048", "btree1024", "btree256", "sj"]].plot() \noverall_duration[["pgm128", "pgm512", "pgm1024", "sj"]].plot() \n'

In [ ]:
inner_index_fetch = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_disk_fetch', aggfunc='median')
inner_index_total_fetch = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_total_bytes_fetched', aggfunc='median')
inner_index_fetch_size = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_disk_fetch_size', aggfunc='median')
print(inner_index_fetch)
print(inner_index_total_fetch)
print(inner_index_fetch_size)

spec.algo_name   flatpgm1024  flatpgm2048  flatpgm256  pgm1024  pgm2048  \
spec.common_key                                                           
1                       3859         2145        9646     3859     2145   
5                       3858         2144        9644     3858     2144   
10                      3858         2144        9643     3858     2144   
50                      3857         2143        9520     3857     2143   
100                     3846         2143        9090     3846     2143   
500                     3383         2058        5532     3383     2058   
1000                    2631         1856        3537     2631     1856   

spec.algo_name   pgm256     sj  
spec.common_key                 
1                  9646  19289  
5                  9644  19289  
10                 9643  19289  
50                 9520  19288  
100                9090  19288  
500                5532  19285  
1000               3537  19282  
spec.algo_name   flatpgm102

In [ ]:
result_checksum = (test_dataframe[['spec.common_key', 'spec.algo_name', 'result.checksum']].sort_values(by=['spec.common_key', 'spec.algo_name'])) #.loc[test_dataframe['spec.common_key']=='10'])
for common_key in sorted(test_dataframe['spec.common_key'].unique()):
    checksums = result_checksum.loc[result_checksum['spec.common_key'] == common_key]
    unique_checksums = checksums['result.checksum'].unique()
    if (len(unique_checksums) == 1):
        print(f"common_key: {common_key} checksum: {unique_checksums} OK")
    else:
        print(f"common_key: {common_key} checksums don't match")
print(result_checksum.loc[result_checksum['spec.common_key'] == 100])
print(result_checksum.loc[result_checksum['spec.common_key'] == 100]['result.checksum'].unique())

common_key: 1 checksum: ['EBE477F42A56C7BAD052D1F07193AD19'] OK
common_key: 5 checksum: ['A2C5CE0B6CB9246BEA6692ECC6AB2A41'] OK
common_key: 10 checksum: ['54B89256C7283C22AE9502BF9A80CE88'] OK
common_key: 50 checksum: ['889FDD67E269850445B46706A0F654F4'] OK
common_key: 100 checksum: ['F0C48A379AB3E1502BA32E58CBFF614A'] OK
common_key: 500 checksum: ['04C3CE8418C53F4CEBE462719E957C38'] OK
common_key: 1000 checksum: ['0954B5E06575449843E15BB63B85353C'] OK
    spec.common_key spec.algo_name                   result.checksum
20              100    flatpgm1024  F0C48A379AB3E1502BA32E58CBFF614A
47              100    flatpgm1024  F0C48A379AB3E1502BA32E58CBFF614A
67              100    flatpgm2048  F0C48A379AB3E1502BA32E58CBFF614A
83              100    flatpgm2048  F0C48A379AB3E1502BA32E58CBFF614A
2               100     flatpgm256  F0C48A379AB3E1502BA32E58CBFF614A
45              100     flatpgm256  F0C48A379AB3E1502BA32E58CBFF614A
50              100        pgm1024  F0C48A379AB3E1502BA32E58